In [5]:
from ultralytics import YOLO
import os
import cv2
import pafy


In [2]:
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# 
# model = YOLO('yolov8n.pt')  # Load model
# 
# # Train the model
# results = model.train(data='VisDrone.yaml', epochs=100, imgsz=640,workers=0)

In [6]:
path_to_weights = 'runs/detect/train/weights/best.pt'
model = YOLO(path_to_weights)

# Open the video file
url = "https://youtu.be/RTwNEfirmks"
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="webm")
cap = cv2.VideoCapture(play.url)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


OSError: ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.